In [ ]:
import os
from typing import TypedDict, Annotated, List, Dict
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables.graph import MermaidDrawMethod
from Ipython.display import display, Imgage
from dotenv import load_dotenv
import requests
import json
from datetime import datetime

In [ ]:
# load environment variables

load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-05-20", temperature=0.3)

Define Agent State and Prompts

In [ ]:
class NewsSummarizerState(TypedDict):
  messages: Annotated[List[HumanMessage | AIMessage], "The messages in the conversation"]
  search_query: str
  search_filter: Dict[str, str]
  raw_news_data: List[Dict]
  processed_articles: List[Dict]
  summary_style: str
  final_summary: str

news_search_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a news search query optimizer. Convert the user's request into an optimized search query for finding relevant news articles. Make it concise and focused."),
    ("human", "User request: {user_input}\nOptimize this into a focused news search query."),
])

news_filter_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a news filtering assistant. Based on the user's request, determine appropriate filters for news search. Return a JSON with 'days' (1-30), 'max_results' (5-20), and 'include_domains' (empty list if no preference)."),
    ("human", "User request: {user_input}\nProvide search filters as JSON."),
])

news_summarize_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert news summarizer. Create a comprehensive summary of the provided news articles in {summary_style} style.

Summary styles:
- brief: 2-3 sentences per article, focus on key points
- detailed: Full paragraph per article with context and implications
- bullet: Key points in bullet format
- executive: Business-focused summary with impact analysis

Structure your response as:
**NEWS SUMMARY - {search_query}**
**Generated on: {timestamp}**

**KEY HIGHLIGHTS:**
[Overall themes and important developments]

**ARTICLE SUMMARIES:**
[Individual article summaries based on style]

**SOURCES:**
[List of sources with publication dates]"""),
    ("human", "Summarize these news articles:\n\n{articles_text}"),
])

